In [2]:
import sqlite3;
import pandas as pd;
import numpy as np;
from datetime import datetime;

In [12]:
# Connect SQLite
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

conn = create_connection("movielens.db")
c = conn.cursor()

In [13]:
# Create tables of movie lens in pandas

path = "../data/"
r_movies = ["movie_id","movie_title","release_date","video_release_date","IMDb_URL","unknown","Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"]
movies = pd.read_csv(path + 'u.item', sep='|', header=None, names=r_movies, encoding = "ISO-8859-1")
movies['release_date'] = pd.to_datetime(movies['release_date'])

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(path + 'u.data', delim_whitespace=True, header=None, names=r_cols, encoding = "ISO-8859-1")

r_users = ["user_id" , "age" , "gender" , "occupation" , "zip_code"]
users = pd.read_csv(path + 'u.user', encoding = "ISO-8859-1", sep='|', names=r_users)

In [14]:
# Create tables from Pandas to SQL
movies.to_sql('MOVIES', conn, if_exists='replace', index=False)
ratings.to_sql('RATINGS', conn, if_exists='replace', index=False)
users.to_sql('USERS', conn, if_exists='replace', index=False)

In [35]:
# JOINS
movie_ratings = c.execute('''
CREATE TABLE IF NOT EXISTS movies_ratings AS
SELECT * FROM movies INNER JOIN ratings 
ON movies.movie_id = ratings.movie_id;
''')
movies_ratings_users = c.execute('''
CREATE TABLE IF NOT EXISTS movies_ratings_users AS
SELECT * FROM movies_ratings INNER JOIN users 
ON movies_ratings.user_id = users.user_id;
''')

In [32]:
# Simple query
c.execute('''SELECT COUNT(*) FROM movies_ratings_users''')
for row in c.fetchall():
    print (row)

(100000,)


In [40]:
# 1) Print a list of the 10 movies that received the most number of ratings.
# most_rated_movies = movies_ratings_users["movie_title"].value_counts(sort=True)
q1 = c.execute('''
SELECT movie_title, COUNT(*) FROM movies_ratings_users
GROUP BY movie_id LIMIT 10;
''')
for row in c.fetchall():
    print (row)

('Toy Story (1995)', 452)
('GoldenEye (1995)', 131)
('Four Rooms (1995)', 90)
('Get Shorty (1995)', 209)
('Copycat (1995)', 86)
('Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 26)
('Twelve Monkeys (1995)', 392)
('Babe (1995)', 219)
('Dead Man Walking (1995)', 299)
('Richard III (1995)', 89)


In [41]:
# 2) Print a list of the 10 movies that received the most number of ratings, sorted by the number of ratings
q2 = c.execute('''
SELECT movie_title, COUNT(*) FROM movies_ratings_users
GROUP BY movie_id ORDER BY COUNT(*) DESC LIMIT 10;
''')
for row in c.fetchall():
    print (row)

('Star Wars (1977)', 583)
('Contact (1997)', 509)
('Fargo (1996)', 508)
('Return of the Jedi (1983)', 507)
('Liar Liar (1997)', 485)
('English Patient, The (1996)', 481)
('Scream (1996)', 478)
('Toy Story (1995)', 452)
('Air Force One (1997)', 431)
('Independence Day (ID4) (1996)', 429)
